In [8]:
import os
from datetime import datetime

from models import FineTuner
from generate_metrics import Metric
from paths import *

In [9]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
## Setting current work directory for relative paths
os.chdir(CURRENT_WD)

In [3]:
model_checkpoint = "bert-base-cased" ## Model to be evaluated
ft = FineTuner(model_name=model_checkpoint) ## For loading saved models or randomly initiliazed models pass the respective arguments in the function call 

In [ ]:
metric = Metric(model_name = model_checkpoint, model = ft.getModel(), tokenizer = ft.getTokenizer(), model_tag= ft.model_tag, model_dir = ft.model_dir)
metric.get_metric(metric = 'crows-pairs', input_file = CP_INPUT_FILE, output_file = CP_OUTPUT_FILE)
metric.get_metric(metric = 'stereoset', input_file = SS_INPUT_FILE, output_file = SS_OUTPUT_FILE,output_dir = SS_OUTPUT_DIR)
metric.get_metric(metric = 'ceat', input_dir = CEAT_INPUT_DIR,output_dir = CEAT_OUTPUT_DIR,generate_new=True)